<style type="text/css" media="screen">
a:link { color:#FFFFFF; text-decoration: none; }
a:visited { color:#FFFFFF; text-decoration: none; }
a:hover { color:#FFFFFF; text-decoration: none; }
a:active { color:#FFFFFF; text-decoration: underline; }
</style>
# <a id="sc" style="text-decoration:none;">Managing HPE Cloud Volumes with API</a>
![](Pictures/hpedevlogo-NB.JPG)

## Part 2 - Manipulate objects in HPE Cloud Volumes

<br>
<br>
<br>
<br>



## If you haven't completed the **part 1 yet**, click the "Back" button below.

 <a href="1-WKSHP-Cloud-Vol-Explore.ipynb" target="New" title="Back: Initialization section"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#FFAD33;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 </br>

## <i class="fas fa-2x fa-cogs" style="color:#FFAD33;"></i>&nbsp;&nbsp;Start the lab

</br>
<p><i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;<b>STEP 1: </b>Reload your environment data</p>

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  
  <p><i class="fas fa-eye"></i>&nbsp;&nbsp;The command set below will reload the environment data saved at the end of the previous notebook. This is needed as the environment variables are not persistent across notebooks.</p>
    <hr>
    <p class="mb-0"><p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the command in the next Cells by clicking - click once per cell - <i class="fas fa-play"></i></p>
 
</div>

In [ ]:
filename="cv-lab.tmp"
TOKEN=`grep TOKEN $HOME/${filename} | awk '{print $2}'`
RepStore=`grep RepStore $HOME/${filename} | awk '{print $2}'`
RepVolume=`grep RepVolume $HOME/${filename} | awk '{print $2}'`
SnapRef=`grep SnapRef $HOME/${filename} | awk '{print $2}'`

</br>

Just to confirm you are ready to continue, test your connection.</br>

In [ ]:
curl -s --location --request GET 'https://demo.cloudvolumes.hpe.com/api/v2/session' \
--header "X-Auth-Token: ${TOKEN}" \
--header 'Content-Type: application/json' | jq

</br>
<p><i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;<b>STEP 2: </b>Create a new block volume</p>

The most recent volume was identified previously. It is now time to clone it. </br>
Once the clone created it will become an independant object that can be presented to a cloud VM for instance.</br>
To achieve this we will use a new verb: "POST" which is required to create an object.</br>
First, we will generate a name for our new volume. Then, we'll clone the volume. The value returned for the second query is the new cloud volume ID.

In [ ]:
uid={{ STDID }}
new_cloudvol_name=cloud_vol-${uid}
echo $RepStore
echo $RepVolume
echo $new_cloudvol_name
echo $SnapRef

In [ ]:
CloudVolumeID=`curl -s --location --request POST "https://demo.cloudvolumes.hpe.com/api/v2/replication_stores/${RepStore}/replica_volumes/${RepVolume}/clone" \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw '{ 
    "data": 
    { 
        "name": "'${new_cloudvol_name}'",
        "snapshot_ref": "'${SnapRef}'",
        "region_id": 1,
        "iops": 300,
        "private_cloud": "vpc-0c722a52c30850c83",
        "existing_cloud_subnet": "10.0.0.0/16"
    }
 }' | jq -r '.data.id'`
 echo "your cloudvolume id is ${CloudVolumeID}"

</br>

Let's verify that the volume exists. </br>
The cURL cpmmand below requests the information for a specific block volume based on its id.

In [ ]:
curl -s --location --request GET "https://demo.cloudvolumes.hpe.com/api/v2/cloud_volumes/${CloudVolumeID}" \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw '' | jq '.'

In [ ]:
echo ${new_cloudvol_name}

</br>
<p><i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;<b>STEP 3: </b>Attach the new volume</p>

</br>

The block volume just created is a lonely element at this time. We need to attach it to a compute element, either a cloud VM or an on-premise compute unit. </br>
For this lab, we have a VM running somewhere in the HPE office of Sophia Antipolis (French riviera for those who need vacation). </br>

First thing, we need to get the initiator target.</br>

In [ ]:
initiator_ip=`curl -s --location --request GET 'https://demo.cloudvolumes.hpe.com/api/v2/initiators' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw '' |  jq -r '. | .[] | .[] | .attributes.ip' `
echo " your initiator IP is ${initiator_ip}"

</br>

It is time now to attach the volume to the target VM using the initiator collected previously.</br>

In [ ]:
curl -s --location --request POST "https://demo.cloudvolumes.hpe.com/api/v2/cloud_volumes/${CloudVolumeID}/attach" \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw '{
    "data":  {
        "initiator_ip": "'${initiator_ip}'"
    }
}'

</br>

Run the command below and look for the section `assigned_initiators` .</br>
This section should start at row 5 more or less.

In [ ]:
curl -s --location --request GET "https://demo.cloudvolumes.hpe.com/api/v2/cloud_volumes/${CloudVolumeID}" \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw '' | jq '.'

</br>
<p><i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;<b>STEP 4: </b>Use the volume with the VM.</p>

In [ ]:
target_name=`curl -s --location --request GET "https://demo.cloudvolumes.hpe.com/api/v2/cloud_volumes/${CloudVolumeID}" \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
| jq -r '.data.attributes.target_name' `
echo ${target_name}

In [ ]:
echo ${new_cloudvol_name}

</br>
<p><i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;<b>STEP 5: </b>Detach and delete the volume</p>


Once the volume is not needed anymore, it can be detached with the query below</br>

In [ ]:
curl -s --location --request POST "https://demo.cloudvolumes.hpe.com/api/v2/cloud_volumes/${CloudVolumeID}/detach" \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw '{
    "data":  {
        "initiator_ip": "'${initiator_ip}'"
    }
}'

</br>

We can now delete the volume as it is not needed anymore.</br>
For that, we will use a new API verb called `DELETE`. The action of this verb is quite explicit.</br>

In [ ]:
curl -s --location --request DELETE "https://demo.cloudvolumes.hpe.com/api/v2/cloud_volumes/${CloudVolumeID}" \
--header "X-auth-Token: ${TOKEN}" \
--header 'Content-Type: application/json'

</br>

Finally, we will look at the status of our volume.</br>
Run the query below and look for the field `mark_for_deletion`.

In [ ]:
curl -s --location --request GET "https://demo.cloudvolumes.hpe.com/api/v2/cloud_volumes/${CloudVolumeID}" \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw '' | jq '.'

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#FFAD33;"></i>&nbsp;&nbsp;Next Steps

Conclusion.

</br>
 <a href="1-WKSHP-Cloud-Vol-Explore.ipynb" target="New" title="Back: Introduction"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#FFAD33;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="3-WKSHP-Cloud-Vol-Conclusion.ipynb#sc" target="New" title="Next:Persistent Volumes"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#FFAD33;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>


<h2>Next&nbsp;&nbsp;&nbsp;&nbsp;<a href="02-Nimble_K8S_RedisDB.ipynb#sc" target="New" title="Next: Introduction"><i class="fas fa-chevron-circle-right" style="color:#FFAD33;"></i></a></h2>